# Jupyter Notebooks Prototypes on how to use Ontology Look-up Service API to navidate the [DFG-Fachsystematik-Ontology](https://github.com/tibonto/DFG-Fachsystematik-Ontology)

## Links
* **DFGFO at TIB Terminology Service (OLS) frontend** https://service.tib.eu/ts4tib/ontologies/dfgfo
* **DFGFO at TIB Terminology Service (OLS) API** https://service.tib.eu/ts4tib/api/ontologies/dfgfo
* **DFGFO root terms at TIB Terminology Service (OLS) API** https://service.tib.eu/ts4tib/api/ontologies/dfgfo/terms/roots

## Notebooks
* extract the DFGFO relations tree - so it can be used in drop-down menus to select an appropriate subject
* extract the DFGFO relations tree from a *non root term*: of the sub-disciplines of Chemistry ([https://github.com/tibonto/dfgfo/31](https://service.tib.eu/ts4tib/ontologies/dfgfo/terms?iri=https%3A%2F%2Fgithub.com%2Ftibonto%2Fdfgfo%2F31&viewMode=All&siblings=true))
* given any DFGFO subject extract all its ancestors


In [1]:
import requests
from urllib.parse import quote_plus
from pprint import pprint

####################################
# imports - global vars - utility functions
####################################

tib_ols_api = 'https://service.tib.eu/ts4tib/api/'
ontology_rootterms = tib_ols_api + 'ontologies/dfgfo/terms/roots'


def url_double_quote(url):
    # NOTE: OLS API IRIs have to be double quoted see https://www.ebi.ac.uk/ols/docs/api
    quoted_url = quote_plus(quote_plus(url)) 
    return quoted_url



In [2]:

####################################
# extract the DFGFO relations tree from OLS API #
# output prints class tree indented further down each level #
# the term['label'] and term['short_form'] (short identifier) are printed # 
# the term['iri'] is ommited but could also be printed/used # 
####################################


def get_terms_from_root(api_url, level):
    # calls OLS API on ontology terms 
    # prints the terms of a given level 
    # and requests the terms parent, recursively 
    apirequest = requests.get(api_url)
    apiresponse = apirequest.json() 
    if '_embedded' in apiresponse.keys():
        terms = apiresponse['_embedded']['terms']
        for term in terms:
            ols_term_url = term['_links']['self']['href']
            # print(ols_term_url)
            print(f"{' ' * level * 2}* {term['label']} - id: {term['short_form']}")
            if 'children' in term['_links'].keys():
                api_children_url = term['_links']['children']['href']
                get_terms_from_root(api_url=api_children_url, level=level+1)
                

get_terms_from_root(api_url=ontology_rootterms, level=0) 



* Life Sciences - id: 2
  * Biology - id: 21
    * Plant Sciences - id: 202
      * Plant Cell and Developmental Biology - id: 202-06
      * Plant Genetics - id: 202-07
      * Plant Physiology - id: 202-04
      * Plant Biochemistry and Biophysics - id: 202-05
      * Ecology and Biodiversity of Plants and Ecosystems - id: 202-02
      * Organismic Interactions, Chemical Ecology and Microbiomes of Plant Systems - id: 202-03
      * Evolution and Systematics of Plants and Fungi - id: 202-01
    * Basic Research in Biology and Medicine - id: 201
      * Bioinformatics and Theoretical Biology - id: 201-07
      * General Genetics and Functional Genome Biology - id: 201-05
      * Developmental Biology - id: 201-06
      * Cell Biology - id: 201-03
      * Structural Biology - id: 201-04
      * Biochemistry - id: 201-01
      * Biophysics - id: 201-02
    * Zoology - id: 203
      * Evolutionary Cell and Developmental Biology (Zoology) - id: 203-06
      * Animal Physiology and Biochemi

In [3]:
####################################
# Extract the relations tree from a *non root term*: ie. start from Chemistry - IRI: https://github.com/tibonto/dfgfo/31 
# does not start '/api/ontologies/dfgfo/terms/roots' but from 'ontologies/dfgfo/terms/TERMIRI'
####################################

def get_children_terms(ols_api, term_iri, level):
    # calls OLS API on an ontology term
    # prints the terms of a given level 
    # and requests the terms children, recursively
    
    term_iri = url_double_quote(url=term_iri) 
    term_api_url = ols_api + '/ontologies/dfgfo/terms/' + term_iri 
    apirequest = requests.get(term_api_url)
    apiresponse = apirequest.json()
    has_children = apiresponse['has_children']
    short_form =  apiresponse['short_form']
    label = apiresponse['label']
    print(f"{' ' * level * 4}* {label} - id: {short_form}")
    if has_children == True:
        children_link = apiresponse['_links']['children']['href']
        apichildrenrequest = requests.get(children_link)
        apichildrenresponse = apichildrenrequest.json()
        children = apichildrenresponse['_embedded']['terms']
        for i, child in enumerate(children):
            get_children_terms(ols_api=ols_api, term_iri=child['iri'], level=level+1)

        
get_children_terms(ols_api='https://service.tib.eu/ts4tib/api',
                   term_iri='https://github.com/tibonto/dfgfo/31',
                   level=0) 

* Chemistry - id: 31


In [ ]:
####################################
# given any DFGFO subject extract all its ancestors # 
# starting at [https://github.com/tibonto/dfgfo/401-03](https://service.tib.eu/ts4tib/ontologies/dfgfo/terms?iri=https%3A%2F%2Fgithub.com%2Ftibonto%2Fdfgfo%2F401-03&viewMode=All&siblings=true) (Joining and Separation Technology)
# extract ancestors
####################################


def get_parent_term(ols_api, term_iri, level):
    # calls OLS API on an ontology term
    # if the term is not a root: requests its parents
    # for each parent (generaly just 1): get its iri, 
    # call the function again (recursevely) with the parent iri
    term_iri = url_double_quote(url=term_iri) 
    term_api_url = ols_api + '/ontologies/dfgfo/terms/' + term_iri 
    apirequest = requests.get(term_api_url)
    apiresponse = apirequest.json()
    is_root = apiresponse['is_root']
    short_form =  apiresponse['short_form']
    label = apiresponse['label']
    print(f"* {' ' * int(level / 4)}{label} - id: {short_form}")
    if is_root == False:
        parents_link = apiresponse['_links']['parents']['href']
        # print(parents_link)
        apiparentrenrequest = requests.get(parents_link)
        apiparentresponse = apiparentrenrequest.json()
        parents = apiparentresponse['_embedded']['terms']
        # print(parents)        
        for i, parent in enumerate(parents):
            parent_iri = parent['iri']
            # print(parent_iri)
            # pprint(parent)
            get_parent_term(ols_api=ols_api, term_iri=parent_iri, level=level-4)

iri = 'https://github.com/tibonto/dfgfo/31'  # Chemistry 31
print(f'\nGet parents of: {iri}')
get_parent_term(ols_api='https://service.tib.eu/ts4tib/api', 
                term_iri=iri, 
                level=12)
print('--------------------------------------')

iri = 'https://github.com/tibonto/dfgfo/401-03'  # Joining and Separation Technology 401-03
print(f'\nGet parents of: {iri}')
get_parent_term(ols_api='https://service.tib.eu/ts4tib/api', 
                term_iri=iri, 
                level=12)
print('--------------------------------------')

iri = 'https://github.com/tibonto/dfgfo/103-02'  # Musicology 103-03
print(f'\nGet parents of: {iri}')
get_parent_term(ols_api='https://service.tib.eu/ts4tib/api', 
                term_iri=iri, 
                level=12)
print('--------------------------------------')

iri = 'https://github.com/tibonto/dfgfo/311'  # Astrophysics and Astronomy 311
print(f'\nGet parents of: {iri}')
get_parent_term(ols_api='https://service.tib.eu/ts4tib/api', 
                term_iri=iri, 
                level=12)
print('--------------------------------------')




Get parents of: https://github.com/tibonto/dfgfo/31
*    Chemistry - id: 31
*   Natural Sciences - id: 3
--------------------------------------

Get parents of: https://github.com/tibonto/dfgfo/401-03
*    Joining and Separation Technology - id: 401-03
*   Production Technology - id: 401
*  Mechanical and Industrial Engineering - id: 41
* Engineering Sciences - id: 4
--------------------------------------

Get parents of: https://github.com/tibonto/dfgfo/103-02
*    Musicology - id: 103-02
*   History - id: 102
*  Humanities - id: 11
* Humanities and Social Sciences - id: 1
--------------------------------------

Get parents of: https://github.com/tibonto/dfgfo/311
*    Astrophysics and Astronomy - id: 311
*   Physics - id: 32
*  Natural Sciences - id: 3
--------------------------------------
